## Data input

In [ ]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb


In [ ]:
# Set up data loading

#Data files
sample_strings = ['Pre_D1','Post_D1','Pre_D2','Post_D2','Pre_D3','Post_D3','Pre_D4','Post_D4']
sample_id_strings = ['65','66','67','68','69','70','71','72']
file_base = './head_and_neck/GSE195832_RAW/GSM58515'
exp_string ='_'
data_file_end = '_matrix.mtx'
barcode_file_end = '_barcodes.tsv'
gene_file_end = '_features.tsv'

In [ ]:
# First data set load & annotation
#Parse Filenames
sample = sample_strings.pop(0)
sample_id = sample_id_strings.pop(0)
data_file = file_base+sample_id+exp_string+sample+data_file_end
barcode_file = file_base+sample_id+exp_string+sample+barcode_file_end
gene_file = file_base+sample_id+exp_string+sample+gene_file_end

#Load data
adata = sc.read(data_file, cache=True)
adata = adata.transpose()
adata.X = adata.X.toarray()

barcodes = pd.read_csv(barcode_file, header=None, sep='\t')
genes = pd.read_csv(gene_file, header=None, sep='\t')

#Annotate data
barcodes.rename(columns={0:'barcode'}, inplace=True)
barcodes.set_index('barcode', inplace=True)
adata.obs = barcodes
adata.obs['sample'] = [sample]*adata.n_obs
adata.obs['status'] = [sample.split("_")[0]]*adata.n_obs
adata.obs['donor'] = [sample.split("_")[1]]*adata.n_obs

genes.rename(columns={0:'gene_id', 1:'gene_symbol'}, inplace=True)
genes.set_index('gene_symbol', inplace=True)
adata.var = genes


In [ ]:
# Loop to load rest of data sets
for i in range(len(sample_strings)):
    #Parse Filenames
    sample = sample_strings[i]
    sample_id = sample_id_strings[i]
    data_file = file_base+sample_id+exp_string+sample+data_file_end
    barcode_file = file_base+sample_id+exp_string+sample+barcode_file_end
    gene_file = file_base+sample_id+exp_string+sample+gene_file_end
    
    #Load data
    adata_tmp = sc.read(data_file, cache=True)
    adata_tmp = adata_tmp.transpose()
    adata_tmp.X = adata_tmp.X.toarray()

    barcodes_tmp = pd.read_csv(barcode_file, header=None, sep='\t')
    genes_tmp = pd.read_csv(gene_file, header=None, sep='\t')
    
    #Annotate data
    barcodes_tmp.rename(columns={0:'barcode'}, inplace=True)
    barcodes_tmp.set_index('barcode', inplace=True)
    adata_tmp.obs = barcodes_tmp
    adata_tmp.obs['sample'] = [sample]*adata_tmp.n_obs
    adata_tmp.obs['status'] = [sample.split("_")[0]]*adata_tmp.n_obs
    adata_tmp.obs['donor'] = [sample.split("_")[1]]*adata_tmp.n_obs
    
    genes_tmp.rename(columns={0:'gene_id', 1:'gene_symbol'}, inplace=True)
    genes_tmp.set_index('gene_symbol', inplace=True)
    adata_tmp.var = genes_tmp
    adata_tmp.var_names_make_unique()

     # Concatenate to main adata object
    adata.var_names_make_unique()
    adata = adata.concatenate(adata_tmp, batch_key='sample_id')
    #adata.var['gene_id'] = adata.var['gene_id-1']
    #adata.var.drop(columns = ['gene_id-1', 'gene_id-0'], inplace=True)
    adata.obs.drop(columns=['sample_id'], inplace=True)
    adata.obs_names = [c.split("-")[0] for c in adata.obs_names]
    adata.obs_names_make_unique(join='_')

In [ ]:
#Annotate the data sets
print(adata.obs['status'].value_counts())
print('')
print(adata.obs['donor'].value_counts())
print('')
print(adata.obs['sample'].value_counts())
print('')

## Pre-Analysis

In [ ]:
# Quality control - calculate QC covariates
adata.obs['n_counts'] = adata.X.sum(1)
adata.obs['log_counts'] = np.log(adata.obs['n_counts'])
adata.obs['n_genes'] = (adata.X > 0).sum(1)
mt_gene_mask = [gene.startswith('MT-') for gene in adata.var_names]
adata.obs['mt_frac'] = adata.X[:, mt_gene_mask].sum(1)/adata.obs['n_counts']

In [ ]:
Ribo_gene_mask = [gene.startswith('RPL-') for gene in adata.var_names]
adata.obs['Ribo_frac'] = adata.X[:, Ribo_gene_mask].sum(1)/adata.obs['n_counts']

In [ ]:
#Data quality summary plots
p1 = sc.pl.scatter(adata, 'n_counts', 'n_genes', color='mt_frac')
p2 = sc.pl.scatter(adata[adata.obs['n_counts']<30000], 'n_counts', 'n_genes', color='mt_frac')

In [ ]:
#Thresholding decision: counts
p3 = sb.distplot(adata.obs['n_counts'], kde=False)
plt.show()

p4 = sb.distplot(adata.obs['n_counts'][adata.obs['n_counts']<1000], kde=False, bins=60)
plt.show()

p5 = sb.distplot(adata.obs['n_counts'][adata.obs['n_counts']>10000], kde=False, bins=60)
plt.show()

In [ ]:
p6 = sb.distplot(adata.obs['n_genes'], kde=False, bins=60)
plt.show()

p7 = sb.distplot(adata.obs['n_genes'][adata.obs['n_genes']<1500], kde=False, bins=60)
plt.show()


In [ ]:
# Filter cells according to identified QC thresholds:
print('Total number of cells: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_counts = 500)
print('Number of cells after min count filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, max_counts = 75000)
print('Number of cells after max count filter: {:d}'.format(adata.n_obs))

adata = adata[adata.obs['mt_frac'] < 0.2]
print('Number of cells after MT filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_genes = 200)
print('Number of cells after gene filter: {:d}'.format(adata.n_obs))


In [ ]:
#Filter genes:
print('Total number of genes: {:d}'.format(adata.n_vars))

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata, min_cells=20)
print('Number of genes after cell filter: {:d}'.format(adata.n_vars))

## Normalization

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

In [ ]:
sc.pp.log1p(adata)

## Batch Effect Correction

In [ ]:
#ComBat batch correction
sc.pp.combat(adata, key='sample')

## Highly Variable Gene 

In [ ]:
sc.pp.highly_variable_genes(adata, flavor='cell_ranger', n_top_genes=4000)
print('\n','Number of highly variable genes: {:d}'.format(np.sum(adata.var['highly_variable'])))

In [ ]:
sc.pl.highly_variable_genes(adata)

## Visualization

In [ ]:
# Calculate the visualizations
sc.pp.pca(adata, n_comps=50, use_highly_variable=True, svd_solver='arpack')
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.pl.umap(adata, color='n_counts')

## Custering

In [ ]:
# Perform clustering - using highly variable genes
sc.tl.leiden(adata, key_added='leiden_r0.4')
sc.tl.leiden(adata, resolution=0.4, key_added='leiden_r0.4', random_state=20)

## Treg markers

In [ ]:
#Treg markers
sc.pl.umap(adata, color=['CD3D','CTLA4','TIGIT','FANK1'], s=50, frameon=False, ncols=4, vmax='p99')

## Cell Fraction Analysis

In [ ]:
tmp = pd.crosstab(ADATA.obs['status'],adata.obs['sample'], normalize='index')
tmp.plot.bar(stacked=False).legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

## Cell Cell Communications

In [ ]:
import squidpy as sq

In [ ]:
rres = sq.gr.ligrec(
    post,
    n_perms=1000,
    cluster_key="leiden_r0.4",
    copy=True,
    seed=0,
    n_jobs=1,
    threshold=0.05,
    use_raw=False,
    transmitter_params={"categories": "ligand"},
    receiver_params={"categories": "receptor"},
    interactions_params={'resources': 'CellPhoneDB'},
    )

In [ ]:
sq.pl.ligrec(rres, source_groups="Treg",target_groups="TAMs", remove_empty_interactions=True, means_range=(0,np.inf), alpha=0.01, swap_axes=True, pvalue_threshold=0.05, remove_nonsig_interactions=True)


## DEG analysis

In [ ]:
sc.tl.rank_genes_groups(adata, 'status', groups=['Post'], reference='Pre', method='t-test', use_raw=True)
sc.pl.rank_genes_groups(adata, groups=['Post'], n_genes=50)

## scFates

In [ ]:
import scFates as scf
import warnings

In [ ]:
adata=adata[:,adata.var.highly_variable]
sc.pp.scale(adata)

In [ ]:
sc.pl.pca(adata,color="leiden_r0.4", legend_loc='on data',cmap="RdBu_r")

In [ ]:
scf.tl.curve(adata,Nodes=10,use_rep="X_pca",ndims_rep=2,)


In [ ]:
scf.pl.graph(adata,basis="pca")


In [ ]:
sc.pl.pca(sc.AnnData(adata.obsm["X_R"],obsm=adata.obsm),color="5",cmap="Reds")


In [ ]:
scf.tl.convert_to_soft(adata,1,1000)

In [ ]:
scf.pl.graph(adata,basis="pca")

In [ ]:
sc.pl.pca(sc.AnnData(adata.obsm["X_R"],obsm=adata.obsm),color="0",cmap="Reds")

In [ ]:
scf.tl.root(adata,"MARCO")

In [ ]:
scf.tl.pseudotime(adata,n_jobs=20,n_map=100,seed=42)

In [ ]:
scf.pl.trajectory(adata,basis="pca",arrows=True,arrow_offset=3)

In [ ]:
scf.tl.linearity_deviation(adata,start_milestone="1",end_milestone="0",n_jobs=20,plot=True,basis="pca")

In [ ]:
scf.pl.linearity_deviation(adata,start_milestone="1",end_milestone="0")

In [ ]:
scf.tl.test_association(adata,n_jobs=20)

In [ ]:
scf.tl.test_association(adata,reapply_filters=True,A_cut=.10)
scf.pl.test_association(adata)

In [ ]:
scf.tl.fit(adata,n_jobs=20)

In [ ]:
scf.pl.single_trend(adata,"Your interested genes",basis="pca",color_exp="k",cmap_seg='RdBu_r')

In [ ]:
scf.tl.cluster(adata,knn=100,metric="correlation")

In [ ]:
adata.var.fit_clusters.unique()

In [ ]:
for c in adata.var["fit_clusters"].unique():
    scf.pl.trends(adata,features=adata.var_names[adata.var.fit_clusters==c],basis="pca")